# Leeds Africa Hackathon
This is a template notebook for the Leeds Africa Hackathon. It also includes some useful Python code to read CMIP6 data and CP4 data

**CMIP6:** 

Historical runs (1850-2014) are in /badc/cmip6/data/CMIP6/CMIP/*institute*/*model*/historical/*variant*/*table_id*/*variable*/*grid*/*version*/
It is likely the *grid* will be the native grid, i.e. gn

*table_id* generally indicates the frequency of the data, e.g. day, 3hr, Amon


Scenario runs are in /badc/cmip6/data/CMIP6/ScenarioMIP/*institute*/*model*/*scenario_name*/*variant*/*table_id*/*variable*/*grid*/*version*/
*scenario_name* is likely to be one of ssp119, ssp126, ssp245, ssp370 or ssp585

The following institutions have data in both historical and ScenarioMIPs:
AS-RCEC, AWI, BCC, CAMS, CAS, CCCR-IITM, CCCma, CMCC, CNRM-CERFACS, CSIRO, CSIRO-ARCCSS, E3SM-Project, EC-Earth-Consortium, FIO-QLNM, HAMMOZ-Consortium, INM, IPSL, KIOST, MIROC, MOHC, MPI-M, MRI, NASA-GISS, NCAR, NCC, NIMS-KMA, NOAA-GFDL, NUIST, THU, UA
       
**CP4:**

4km data are in /badc/impala/data/explicit-4km/*present_or_future*>/*1hr_or_mon*/*variable*/

In [9]:
!ls /badc/cmip6/data/CMIP6/ScenarioMIP/*/*/ssp585/*/day/tas/gn/

/badc/cmip6/data/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/ssp585/r1i1p1f1/day/tas/gn/:
files  latest  v20200902

/badc/cmip6/data/CMIP6/ScenarioMIP/AWI/AWI-CM-1-1-MR/ssp585/r1i1p1f1/day/tas/gn/:
files  latest  v20190529

/badc/cmip6/data/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp585/r1i1p1f1/day/tas/gn/:
files  latest  v20190318

/badc/cmip6/data/CMIP6/ScenarioMIP/CAMS/CAMS-CSM1-0/ssp585/r2i1p1f1/day/tas/gn/:
files  latest  v20191106

/badc/cmip6/data/CMIP6/ScenarioMIP/CAS/FGOALS-g3/ssp585/r1i1p1f1/day/tas/gn/:
files  latest  v20190819

/badc/cmip6/data/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp585/r1i1p1f1/day/tas/gn/:
files  latest  v20200915

/badc/cmip6/data/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r10i1p1f1/day/tas/gn/:
files  latest  v20190429

/badc/cmip6/data/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r10i1p2f1/day/tas/gn/:
files  latest  v20190429

/badc/cmip6/data/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp585/r11i1p1f1/day/tas/gn/:
files  latest  v20190429

/badc/cmip6/data/CMIP6/ScenarioMIP/CCCma/CanE

In [10]:
!ls /badc/cmip6/data/CMIP6/CMIP/*/*/historical

/badc/cmip6/data/CMIP6/CMIP/AS-RCEC/TaiESM1/historical:
00README_catalogue_and_licence.txt  r1i1p1f1

/badc/cmip6/data/CMIP6/CMIP/AWI/AWI-CM-1-1-MR/historical:
r1i1p1f1  r2i1p1f1  r3i1p1f1  r4i1p1f1	r5i1p1f1

/badc/cmip6/data/CMIP6/CMIP/AWI/AWI-ESM-1-1-LR/historical:
r1i1p1f1

/badc/cmip6/data/CMIP6/CMIP/BCC/BCC-CSM2-MR/historical:
00README_catalogue_and_licence.txt  r1i1p1f1  r2i1p1f1	r3i1p1f1

/badc/cmip6/data/CMIP6/CMIP/BCC/BCC-ESM1/historical:
00README_catalogue_and_licence.txt  r1i1p1f1  r2i1p1f1	r3i1p1f1

/badc/cmip6/data/CMIP6/CMIP/CAMS/CAMS-CSM1-0/historical:
r1i1p1f1  r1i1p1f2  r2i1p1f1

/badc/cmip6/data/CMIP6/CMIP/CAS/CAS-ESM2-0/historical:
00README_catalogue_and_licence.txt  r1i1p1f1  r2i1p1f1	r3i1p1f1

/badc/cmip6/data/CMIP6/CMIP/CAS/FGOALS-f3-L/historical:
00README_catalogue_and_licence.txt  r1i1p1f1  r2i1p1f1	r3i1p1f1

/badc/cmip6/data/CMIP6/CMIP/CAS/FGOALS-g3/historical:
r1i1p1f1  r2i1p1f1  r3i1p1f1  r4i1p1f1	r5i1p1f1

/badc/cmip6/data/CMIP6/CMIP/CCCR-IITM/IITM-ESM/histo

In [ ]:
!ls /badc/impala/data/explicit-4km/future/mon/precipitation

In [ ]:
#-----------------------------------------------------------------------------------------------------
# here is an example reading daily tas for the Met office HadGEM3-GC31-LL model from the CMIP6 archive
#-----------------------------------------------------------------------------------------------------
from read_cmip6 import *
import datetime
tas=read_cmip6('ScenarioMIP','MOHC','HadGEM3-GC31-LL','ssp585','r1i1p1f3','day','tas','gn','v20200114','20150101-20491230')
print(tas)
time_coord = tas.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])

In [1]:
#-----------------------------------------------------------------------------------------------------
# Here is an example reading present hourly tas for the CP4 archive
# this only reads a few months as it takes a long time to read all 10 years
#-----------------------------------------------------------------------------------------------------
from read_cp4 import *
import datetime
start_year=1997
start_month=3
end_year=1997
end_month=6
tas=read_cp4('1hr','tas', start_year, start_month, end_year, end_month)
print(tas)
time_coord = tas.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])

reading 1997/03 to 1997/06
air_temperature / (K)               (time: 2880; latitude: 2100; longitude: 2000)
     Dimension coordinates:
          time                           x               -                -
          latitude                       -               x                -
          longitude                      -               -                x
     Scalar coordinates:
          height: 1.5 m
     Attributes:
          CDI: Climate Data Interface version 1.9.5 (http://mpimet.mpg.de/cdi)
          CDO: Climate Data Operators version 1.9.5 (http://mpimet.mpg.de/cdo)
          Conventions: CF-1.5
          NCO: 4.7.3
          PP_LBPROC: 0
          PP_LBTIM: 12
          PP_MEAN_PERIOD: 0.0
          STASH: m01s03i236
          contact: cath.senior@metoffice.gov.uk
          copyright: (C) Crown Copyright, 2015 the Met Office
          description: hourly instantaneous data
          history: Source data files originally generated using cmorize v0.1.5b and subsequently.

In [2]:
#-----------------------------------------------------------------------------------------------------
# here is an example reading all 10 future years of monthly precip for the CP4 archive
#-----------------------------------------------------------------------------------------------------
from read_cp4 import *
import datetime
start_year=2095
start_month=3
end_year=2105
end_month=2
pr=read_cp4('mon','pr', start_year, start_month, end_year, end_month)
print(pr)
time_coord = pr.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])

reading 2095/03 to 2105/02
precipitation_flux / (kg m-2 s-1)   (time: 120; latitude: 2100; longitude: 2000)
     Dimension coordinates:
          time                           x              -                -
          latitude                       -              x                -
          longitude                      -              -                x
     Attributes:
          CDI: Climate Data Interface version 1.9.3 (http://mpimet.mpg.de/cdi)
          CDO: Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/cdo)
          Conventions: CF-1.5
          NCO: netCDF Operators version 4.7.5 (Homepage = http://nco.sf.net, Code = h...
          PP_LBPROC: 128
          PP_LBTIM: 122
          PP_MEAN_PERIOD: 1.0
          STASH: m01s04i203
          contact: cath.senior@metoffice.gov.uk
          copyright: (C) Crown Copyright, 2015 the Met Office
          description: monthly means calculated from hourly means
          history: Source data files originally generated usin